# Transfer Learning using CNN VGG16 with SVM
### For feature extraction

In [62]:
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Input, Lambda, Dense, Flatten

IMAGE_SIZE = [224, 224]
base_model = VGG16(input_shape=IMAGE_SIZE + [3],weights='imagenet',include_top = False)

In [63]:
base_model.input

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_2')>

In [64]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(base_model.output)
prediction = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=prediction)
model.summary()

In [ ]:
import os
import numpy as np

def get_features(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    flatten = model.predict(x)
    return list(flatten[0])

features, labels = [], []

# Loop into the directory of images and extract features and labels
for image_path in os.listdir('lie'):
    features.append(get_features('lie/'+image_path))
    labels.append(1)

for image_path in os.listdir('truth'):
    features.append(get_features('truth/'+image_path))
    labels.append(0)

In [76]:
import pandas as pd

features_df = pd.DataFrame(features)
labels_df = pd.DataFrame(labels)

In [77]:
features_df.head()
#len(features_df)

,0,1
0,0.859760,1.402402e-01
1,0.989146,1.085386e-02
2,0.853514,1.464856e-01
3,0.983571,1.642930e-02
4,0.999999,8.500992e-07


In [78]:
labels_df.head()
#labels_df.tail()

,0
0,1
1,1
2,1
3,1
4,1


In [79]:
features_df.insert(2,'target',labels_df)

In [80]:
features_df.head()

,0,1,target
0,0.859760,1.402402e-01,1
1,0.989146,1.085386e-02,1
2,0.853514,1.464856e-01,1
3,0.983571,1.642930e-02,1
4,0.999999,8.500992e-07,1


In [81]:
features_df = features_df.sample(frac=1)

In [82]:
features_df.head()

,0,1,target
1304,1.000000,5.640984e-08,0
667,0.011131,9.888690e-01,0
1580,0.019195,9.808052e-01,0
670,0.004148,9.958516e-01,0
329,0.000095,9.999048e-01,1


In [83]:
features_x = features_df.drop("target", axis = 1)
target_y = features_df["target"]

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split( features_x,
                                                     target_y,
                                                     test_size=0.30)
                                         
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X_train, y_train)

predicted = clf.predict(X_test)

# get the accuracy
print(accuracy_score(y_test, predicted))

0.7213930348258707
